In [27]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
#FreightRates cədvəli bütün mövcud kuryerləri, hər bir fərdi zolaq üçün çəki boşluqlarını və əlaqəli tarifləri təsvir edir.
#PlantPorts cədvəli real dünyada anbarlar və göndərmə limanları arasında icazə verilən əlaqələri təsvir edir
#ProductsPerPlant cədvəlində dəstəklənən bütün anbar-məhsul birləşmələri sadalanır.
#WhCapacities gündə sifarişlərin sayı ilə ölçülən anbar tutumlarını sadalayır
#WhCosts hər vahid üçün dollarla ölçülən verilmiş anbarda məhsulların saxlanması ilə bağlı xərcləri müəyyən edir.
#tpt_day_cnt catdırılma gununun sayını göstərir
#WhCapacities sifarişlərin sayına uyğundur.
#Məsələn, deyək ki, Müştəri 1 10 vahid X, Müştəri 2 20 vahid Y tələb edir. Sifarişlərin ümumi sayı 2-dir, beləliklə, "whCapacity"də ümumi tutum 2-dir.
# the product ID that needs to be shipped
#, the destination port
#, unit quantity (for cost) and unit weight (for weight constraints)

In [29]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [30]:
mainFile = pd.ExcelFile("Supply chain logisitcs problem.xlsx")

In [31]:
df_dict = {}

for names in mainFile.sheet_names:
    globals()[names] = mainFile.parse(names)
    df_dict[names] = globals()[names]
   
df_dict.keys()

dict_keys(['OrderList', 'FreightRates', 'WhCosts', 'WhCapacities', 'ProductsPerPlant', 'VmiCustomers', 'PlantPorts'])

In [33]:
for df_name, df in df_dict.items():
    print(df_name, '- shape:', df.shape)
    duplicate_count = df.duplicated().sum()
    missing_values_count = df.isnull().sum().sum()
    
    if duplicate_count > 0 or missing_values_count > 0:
        print(f">>>>{df_name} - duplicates: {duplicate_count}; missing values: {missing_values_count}")

OrderList - shape: (9215, 14)
FreightRates - shape: (1540, 11)
>>>>FreightRates - duplicates: 3; missing values: 0
WhCosts - shape: (19, 2)
WhCapacities - shape: (19, 2)
ProductsPerPlant - shape: (2036, 2)
VmiCustomers - shape: (14, 2)
PlantPorts - shape: (22, 2)


In [9]:
FreightRates = FreightRates.drop_duplicates()
df_dict['FreightRates'] = FreightRates

In [10]:
df_dict

{'OrderList':           Order ID Order Date Origin Port Carrier  TPT Service Level  \
 0     1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
 1     1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
 2     1.447139e+09 2013-05-26      PORT09   V44_3    1           CRF   
 3     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 4     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 ...            ...        ...         ...     ...  ...           ...   
 9210  1.447305e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9211  1.447319e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9212  1.447322e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9213  1.447145e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9214  1.447328e+09 2013-05-26      PORT04  V444_1    1           DTD   
 
       Ship ahead day count  Ship Late Day count           Customer  \
 0                        3           

In [11]:
mainFile

In [12]:
for df_name, df in df_dict.items():
    df.columns = [col.strip().replace(' ', '_').replace('/', '_').upper() for col in df.columns]

In [14]:
orderList = df_dict['OrderList']  
# order list datsını FreightRates və WhCosts datası ilə müvafiq sütunları birləşdiririk
orderList = orderList.merge(df_dict['FreightRates'], left_on=['CARRIER', 'ORIGIN_PORT', 'DESTINATION_PORT'],
                            right_on=['CARRIER', 'ORIG_PORT_CD', 'DEST_PORT_CD'], how='left')
orderList = orderList.merge(df_dict['WhCosts'], left_on='PLANT_CODE', right_on='WH', how='left')
# Vahid miqdarını göndərmə dərəcəsi ilə vuraraq və saxlama dəyərini əlavə etməklə costu hesablayırıq
orderList['COST'] = (orderList['UNIT_QUANTITY'] * orderList['RATE']) + (orderList['UNIT_QUANTITY'] * orderList['COST_UNIT'])
orderList = orderList.dropna() 
orderList['ORDER_DATE'] = pd.to_datetime(orderList['ORDER_DATE'])
productsPerPlant = df_dict['ProductsPerPlant']
productsPerPlant = productsPerPlant.dropna()  
# dicti update edirik 
df_dict['OrderList'] = orderList
df_dict['ProductsPerPlant'] = productsPerPlant

orderList

,ORDER_ID,ORDER_DATE,ORIGIN_PORT,CARRIER,TPT,SERVICE_LEVEL,SHIP_AHEAD_DAY_COUNT,SHIP_LATE_DAY_COUNT,CUSTOMER,PRODUCT_ID,...,MAX_WGH_QTY,SVC_CD,MINIMUM_COST,RATE,MODE_DSC,TPT_DAY_CNT,CARRIER_TYPE,WH,COST_UNIT,COST
854,1.447385e+09,2013-05-26,PORT09,V444_0,0,DTP,0,0,V55555_4,1692724,...,5000.00,DTP,31.2784,12.2784,GROUND,0.0,V88888888_0,PLANT16,1.919808,4713.804893
855,1.447385e+09,2013-05-26,PORT09,V444_0,0,DTP,0,0,V55555_4,1692724,...,5000.00,DTD,31.2784,13.2784,GROUND,0.0,V88888888_0,PLANT16,1.919808,5045.804893
856,1.447338e+09,2013-05-26,PORT09,V444_0,0,DTP,0,0,V55555_4,1692724,...,5000.00,DTP,31.2784,12.2784,GROUND,0.0,V88888888_0,PLANT16,1.919808,5082.958288
857,1.447338e+09,2013-05-26,PORT09,V444_0,0,DTP,0,0,V55555_4,1692724,...,5000.00,DTD,31.2784,13.2784,GROUND,0.0,V88888888_0,PLANT16,1.919808,5440.958288
858,1.447407e+09,2013-05-26,PORT09,V444_0,0,DTP,3,0,V555555_6,1692722,...,5000.00,DTP,31.2784,12.2784,GROUND,0.0,V88888888_0,PLANT16,1.919808,5807.066871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209397,1.447328e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683424,...,99.99,DTD,7.8044,0.0804,AIR,1.0,V888888883_1,PLANT03,0.517502,189.534900
209398,1.447328e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683424,...,299.99,DTD,11.2272,0.0792,AIR,1.0,V888888883_1,PLANT03,0.517502,189.154500
209399,1.447328e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683424,...,499.99,DTD,31.2672,0.0780,AIR,1.0,V888888883_1,PLANT03,0.517502,188.774100
209400,1.447328e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683424,...,999.99,DTD,49.4272,0.0760,AIR,1.0,V888888883_1,PLANT03,0.517502,188.140100


In [15]:
for df_name, df in df_dict.items():
    print(f'Dataframe: {df_name} >>>')
    print('Columns:', df.columns.values)

Dataframe: OrderList >>>
Columns: ['ORDER_ID' 'ORDER_DATE' 'ORIGIN_PORT' 'CARRIER' 'TPT' 'SERVICE_LEVEL'
 'SHIP_AHEAD_DAY_COUNT' 'SHIP_LATE_DAY_COUNT' 'CUSTOMER' 'PRODUCT_ID'
 'PLANT_CODE' 'DESTINATION_PORT' 'UNIT_QUANTITY' 'WEIGHT' 'ORIG_PORT_CD'
 'DEST_PORT_CD' 'MINM_WGH_QTY' 'MAX_WGH_QTY' 'SVC_CD' 'MINIMUM_COST'
 'RATE' 'MODE_DSC' 'TPT_DAY_CNT' 'CARRIER_TYPE' 'WH' 'COST_UNIT' 'COST']
Dataframe: FreightRates >>>
Columns: ['CARRIER' 'ORIG_PORT_CD' 'DEST_PORT_CD' 'MINM_WGH_QTY' 'MAX_WGH_QTY'
 'SVC_CD' 'MINIMUM_COST' 'RATE' 'MODE_DSC' 'TPT_DAY_CNT' 'CARRIER_TYPE']
Dataframe: WhCosts >>>
Columns: ['WH' 'COST_UNIT']
Dataframe: WhCapacities >>>
Columns: ['PLANT_ID' 'DAILY_CAPACITY']
Dataframe: ProductsPerPlant >>>
Columns: ['PLANT_CODE' 'PRODUCT_ID']
Dataframe: VmiCustomers >>>
Columns: ['PLANT_CODE' 'CUSTOMERS']
Dataframe: PlantPorts >>>
Columns: ['PLANT_CODE' 'PORT']


In [19]:
orderList.to_csv("orderList.csv")

In [20]:
df_dict['FreightRates'].to_csv("FreightRates.csv")

In [21]:
df_dict['WhCapacities'].to_csv("WhCapacities.csv")

In [22]:
df_dict['ProductsPerPlant'].to_csv("ProductsPerPlant.csv")

In [23]:
df_dict['WhCosts'].to_csv("WhCosts.csv")

In [26]:
df_dict['PlantPorts'].to_csv("PlantPorts.csv")